In [1]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

In [2]:
data_customers = "/Users/ashwin/Applications/Master/ml-sys/data/customers.csv"
data_non_customers = "/Users/ashwin/Applications/Master/ml-sys/data/noncustomers.csv"
usage_actions = "/Users/ashwin/Applications/Master/ml-sys/data/usage_actions.csv"
df1, df2, df3 = pd.read_csv(data_customers), pd.read_csv(data_non_customers), pd.read_csv(usage_actions)


In [3]:
df1.head()

,CLOSEDATE,MRR,ALEXA_RANK,EMPLOYEE_RANGE,INDUSTRY,id
0,2019-06-20,290.00,309343,201 to 1000,Other,199
1,2019-03-22,392.54,16000001,51 to 200,NaN,147
2,2020-01-08,-61.15,60502,1001 to 10000,HIGHER_EDUCATION,118
3,2019-09-19,400.00,3575321,201 to 1000,CONSUMER_SERVICES,126
4,2019-01-27,209.98,273063,26 to 50,Technology - Software,174


In [4]:
df2.head()

,ALEXA_RANK,EMPLOYEE_RANGE,INDUSTRY,id
0,4489573.0,11 to 25,NaN,4916
1,4027013.0,6 to 10,NaN,3489
2,16000001.0,26 to 50,COMPUTER_SOFTWARE,2813
3,814822.0,1001 to 10000,HOSPITAL_HEALTH_CARE,727
4,16000001.0,26 to 50,NaN,2166


In [5]:
df3.head()

,WHEN_TIMESTAMP,ACTIONS_CRM_CONTACTS,ACTIONS_CRM_COMPANIES,ACTIONS_CRM_DEALS,ACTIONS_EMAIL,USERS_CRM_CONTACTS,USERS_CRM_COMPANIES,USERS_CRM_DEALS,USERS_EMAIL,id
0,2020-06-01 00:00:00.000,278,0,34,0,2,0,2,0,616
1,2020-04-20 00:00:00.000,0,0,0,0,0,0,0,0,3059
2,2019-04-29 00:00:00.000,210,1,17,11,3,1,4,1,2813
3,2020-02-24 00:00:00.000,1,1,1,0,1,1,1,0,5076
4,2020-02-17 00:00:00.000,9,9,7,0,1,1,1,0,3983


In [6]:
len(df1), len(df2), len(df3)

(200, 5003, 25387)

In [7]:
customers = df1.copy()
noncustomers = df2.copy()
usage = df3.copy()


In [8]:
for name, df in [("customers", customers),
                 ("noncustomers", noncustomers),
                 ("usage", usage)]:
    print(f"\n=== {name.upper()} ===")
    print("Shape:", df.shape)
    print("\nDtypes:\n", df.dtypes)
    print("\nMissing %:\n", df.isna().mean().sort_values(ascending=False).head(10))



=== CUSTOMERS ===
Shape: (200, 6)

Dtypes:
 CLOSEDATE          object
MRR               float64
ALEXA_RANK          int64
EMPLOYEE_RANGE     object
INDUSTRY           object
id                  int64
dtype: object

Missing %:
 INDUSTRY          0.645
EMPLOYEE_RANGE    0.010
CLOSEDATE         0.000
MRR               0.000
ALEXA_RANK        0.000
id                0.000
dtype: float64

=== NONCUSTOMERS ===
Shape: (5003, 4)

Dtypes:
 ALEXA_RANK        float64
EMPLOYEE_RANGE     object
INDUSTRY           object
id                  int64
dtype: object

Missing %:
 INDUSTRY          0.744553
EMPLOYEE_RANGE    0.106336
ALEXA_RANK        0.022786
id                0.000000
dtype: float64

=== USAGE ===
Shape: (25387, 10)

Dtypes:
 WHEN_TIMESTAMP           object
ACTIONS_CRM_CONTACTS      int64
ACTIONS_CRM_COMPANIES     int64
ACTIONS_CRM_DEALS         int64
ACTIONS_EMAIL             int64
USERS_CRM_CONTACTS        int64
USERS_CRM_COMPANIES       int64
USERS_CRM_DEALS           int64
USERS_EMAI

In [9]:
customers = customers.copy()
noncustomers = noncustomers.copy()

customers["is_customer"] = 1
noncustomers["is_customer"] = 0

# pick the common columns + label
base = pd.concat(
    [
        customers.assign(source="customer"),
        noncustomers.assign(source="noncustomer")
    ],
    ignore_index=True
)

base.head()


,CLOSEDATE,MRR,ALEXA_RANK,EMPLOYEE_RANGE,INDUSTRY,id,is_customer,source
0,2019-06-20,290.00,309343.0,201 to 1000,Other,199,1,customer
1,2019-03-22,392.54,16000001.0,51 to 200,NaN,147,1,customer
2,2020-01-08,-61.15,60502.0,1001 to 10000,HIGHER_EDUCATION,118,1,customer
3,2019-09-19,400.00,3575321.0,201 to 1000,CONSUMER_SERVICES,126,1,customer
4,2019-01-27,209.98,273063.0,26 to 50,Technology - Software,174,1,customer


In [10]:
base["is_customer"].value_counts(normalize=True)


is_customer
0    0.961561
1    0.038439
Name: proportion, dtype: float64

In [11]:
usage["WHEN_TIMESTAMP"] = pd.to_datetime(usage["WHEN_TIMESTAMP"])

# choose some numeric columns to aggregate
usage_cols_to_sum = [
    "ACTIONS_CRM_CONTACTS", "ACTIONS_CRM_COMPANIES",
    "ACTIONS_CRM_DEALS", "ACTIONS_EMAIL"
]
usage_cols_to_max = [
    "USERS_CRM_CONTACTS", "USERS_CRM_COMPANIES",
    "USERS_CRM_DEALS", "USERS_EMAIL"
]

agg_dict = {c: "sum" for c in usage_cols_to_sum}
agg_dict.update({c: "max" for c in usage_cols_to_max})

usage_agg = (
    usage
    .groupby("id")
    .agg(agg_dict)
    .reset_index()
)

usage_agg.head()


,id,ACTIONS_CRM_CONTACTS,ACTIONS_CRM_COMPANIES,ACTIONS_CRM_DEALS,ACTIONS_EMAIL,USERS_CRM_CONTACTS,USERS_CRM_COMPANIES,USERS_CRM_DEALS,USERS_EMAIL
0,1,14845,1889,5253,70,10,8,10,2
1,2,4970,798,790,247,4,4,4,3
2,3,43467,8219,18797,498,26,17,13,4
3,4,54329,77,867,0,11,2,10,0
4,5,5172,1450,3551,70,9,6,9,2


In [12]:
data = base.merge(usage_agg, on="id", how="left")

data.shape, data.isna().mean().sort_values(ascending=False).head(10)


((5203, 16),
 CLOSEDATE                0.961561
 MRR                      0.961561
 INDUSTRY                 0.740727
 ACTIONS_CRM_CONTACTS     0.313857
 ACTIONS_CRM_COMPANIES    0.313857
 ACTIONS_CRM_DEALS        0.313857
 ACTIONS_EMAIL            0.313857
 USERS_CRM_CONTACTS       0.313857
 USERS_CRM_COMPANIES      0.313857
 USERS_CRM_DEALS          0.313857
 dtype: float64)

In [13]:
fig = px.bar(
    data["is_customer"].value_counts().rename_axis("is_customer").reset_index(name="count"),
    x="is_customer",
    y="count",
    text="count",
    title="Customer vs Non-Customer Count"
)
fig.update_traces(textposition="outside")
fig.show()


In [14]:
numeric_cols = [
    c for c in data.columns
    if pd.api.types.is_numeric_dtype(data[c]) and c not in ["is_customer"]
]

numeric_cols[:10]


['MRR',
 'ALEXA_RANK',
 'id',
 'ACTIONS_CRM_CONTACTS',
 'ACTIONS_CRM_COMPANIES',
 'ACTIONS_CRM_DEALS',
 'ACTIONS_EMAIL',
 'USERS_CRM_CONTACTS',
 'USERS_CRM_COMPANIES',
 'USERS_CRM_DEALS']

In [16]:
for col in ["MRR", "ACTIONS_CRM_DEALS", "ACTIONS_EMAIL"]:
    if col in data.columns:
        fig = px.histogram(
            data,
            x=col,
            color="is_customer",
            barmode="overlay",
            nbins=50,
            opacity=0.6,
            marginal="box",  # adds a boxplot on the side
            title=f"Distribution of {col} by is_customer"
        )
        fig.show()


In [17]:
def plot_conversion_rate(col):
    if col not in data.columns:
        return
    tmp = (
        data.groupby(col)["is_customer"]
        .mean()
        .reset_index()
        .rename(columns={"is_customer": "conversion_rate"})
        .sort_values("conversion_rate", ascending=False)
    )

    fig = px.bar(
        tmp,
        x=col,
        y="conversion_rate",
        title=f"Conversion rate by {col}",
        text="conversion_rate"
    )
    fig.update_traces(texttemplate="%{text:.2f}", textposition="outside")
    fig.update_layout(xaxis_tickangle=-45)
    fig.show()

for col in ["EMPLOYEE_RANGE", "INDUSTRY"]:
    plot_conversion_rate(col)


In [18]:
corr_cols = [c for c in numeric_cols if c in data.columns] + ["is_customer"]
corr = data[corr_cols].corr()

fig = px.imshow(
    corr,
    text_auto=".2f",
    aspect="auto",
    title="Correlation matrix"
)
fig.show()
